Pleaes run preprocess_Data module first before run this module.

Author: xzhang867

## Install Packages

In [ ]:
!pip install keras-tuner

## Import Packages

In [2]:
import pandas as pd

import tensorflow as tf
import keras_tuner as kt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from google.colab import drive

from tensorflow import keras


## Map Google Drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Build CNN Model

### Set Common Parameters



In [4]:
nb_train_samples =2312
nb_validation_samples = 663
epochs = 10
img_width, img_height = 256, 256
batch_size = 16

### Generate Training Dataset

In [5]:
train_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/train'

train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 2312 images belonging to 5 classes.


### Generate Validation Dataset

In [6]:
validation_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/validation'

validation_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = validation_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 663 images belonging to 5 classes.


### Generate Test Dataset

In [7]:
test_data_dir = '/content/drive/MyDrive/practicum/data_image_classification/test'

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
	test_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='categorical')

Found 328 images belonging to 5 classes.


### Prepare CNN Model

In [8]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [9]:
def model_builder(hp):
  '''
  Args:
    hp - Keras tuner object
  '''
  # initiate CNN model
  model = Sequential()
  model.add(Conv2D(32, (2, 2), input_shape=input_shape))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # add layer
  model.add(Conv2D(32, (2, 2)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # add layer
  model.add(Conv2D(64, (2, 2)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(hp.Choice('units', [8, 16, 32, 64, 128, 256])))
  model.add(Activation('relu'))

  # add dropout to avoid overfitting
  model.add(Dropout(0.5))

  # output layer contains 5 neuron
  model.add(Dense(5))
  model.add(Activation('softmax'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(loss='categorical_crossentropy',
			optimizer= keras.optimizers.RMSprop(learning_rate=hp_learning_rate),
			metrics=['accuracy'])
  
  return model

### Tune the Hyperparameters

In [10]:
tuner = kt.RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=10)

tuner.search(train_generator, epochs=10, validation_data=validation_generator)

best_model = tuner.get_best_models()[0]

Trial 10 Complete [00h 32m 45s]
val_accuracy: 0.7375565767288208

Best val_accuracy So Far: 0.7888386249542236
Total elapsed time: 03h 10m 51s
INFO:tensorflow:Oracle triggered exit


## Save Model

## Evaluate Model (Test Data)

In [11]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 255, 255, 32)      416       
                                                                 
 activation (Activation)     (None, 255, 255, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      4128      
                                                                 
 activation_1 (Activation)   (None, 126, 126, 32)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                    

In [ ]:
evaluate = best_model.evaluate(test_generator)
print(evaluate)

14/21 [===================>..........] - ETA: 32s - loss: 0.4838 - accuracy: 0.8036

## Save the Model

In [ ]:
best_model.save("best_model.h5")